In [29]:
import json
import pandas as pd
from datetime import datetime




In [30]:
# Load the JSON data
with open('./Fit/All data/raw_com.google.step_count.delta_com.google.and.json', 'r') as file:
    data = json.load(file)

# Extract the data points
data_points = data['Data Points']



In [31]:
# Empty list to store the transformed data
transformed_data = []

for point in data_points:
    steps = point['fitValue'][0]['value']['intVal']
    start_time = int(point['startTimeNanos'])
    end_time = int(point['endTimeNanos'])
    
    # Convert nanoseconds to datetime
    start_datetime = datetime.fromtimestamp(start_time / 1e9)
    end_datetime = datetime.fromtimestamp(end_time / 1e9)
    
    transformed_data.append({
        'steps': steps,
        'start_time': start_datetime,
        'end_time': end_datetime
    })

df = pd.DataFrame(transformed_data)


In [32]:
# Aggregate by day
df['date'] = df['start_time'].dt.date
daily_steps = df.groupby('date')['steps'].sum().reset_index()

print(daily_steps)


          date   steps
0   2024-06-08    3615
1   2024-06-09    2491
2   2024-06-10   18051
3   2024-06-11   13049
4   2024-06-12   16118
5   2024-06-13   13301
6   2024-06-14   16142
7   2024-06-15    1950
8   2024-06-16    8883
9   2024-06-17   18681
10  2024-06-18    9249
11  2024-06-19    9895
12  2024-06-20   11682
13  2024-06-21    2262
14  2024-06-22    2939
15  2024-06-23    9574
16  2024-06-24   16989
17  2024-06-25    9963
18  2024-06-26    9079
19  2024-06-27   16190
20  2024-06-28   13895
21  2024-06-29    2641
22  2024-06-30    9679
23  2024-07-01    8469
24  2024-07-02    8051
25  2024-07-03    8659
26  2024-07-04   12646
27  2024-07-05    7947
28  2024-07-06    4552
29  2024-07-07    2105
30  2024-07-08    9542
31  2024-07-09   15188
32  2024-07-10    8000
33  2024-07-11  196007


# I noticed some inconsistencies in the day 2024-07-11, I believe that I didn't do that amount of steps.
# "33  2024-07-11  196007"
# I will need to fix this and copy mannually from the app, as it's just one data


# I will separate this data into weeks before merging with the football performance data. This will help in making the analysis more structured and easier to manage.

In [33]:
df_steps.loc[33, 'steps'] = 8720
print(df_steps.loc[33])

date            2024-07-11 00:00:00
steps                          8720
week                              5
weekly_steps                 243341
Name: 33, dtype: object


In [37]:
# Assuming you already have the daily_steps DataFrame
daily_steps['date'] = pd.to_datetime(daily_steps['date'])

# Subtract 4 days to shift the week start to Friday
daily_steps['adjusted_date'] = daily_steps['date'] - pd.Timedelta(days=4)

# Sort by adjusted_date to ensure weeks are calculated correctly
daily_steps = daily_steps.sort_values('adjusted_date')

# Add a 'week' column based on adjusted_date
daily_steps['week'] = daily_steps['adjusted_date'].dt.isocalendar().week

# Group by week and sum the steps
weekly_steps_sum = daily_steps.groupby('week')['steps'].sum().reset_index()

# Merge the weekly sums back into the original DataFrame
df_steps = pd.merge(daily_steps, weekly_steps_sum, on='week', suffixes=('', '_weekly'))

# Rename columns for clarity
df_steps.rename(columns={'steps_weekly': 'weekly_steps'}, inplace=True)

# Enumerate the week column starting from 1
df_steps['week'] = df_steps['week'].rank(method='dense').astype(int)

# Drop the adjusted_date column as it's no longer needed
df_steps.drop(columns=['adjusted_date'], inplace=True)

print(df_steps)


KeyError: 'date'